In [1]:
import numpy as np
import cv2
import time
import pyautogui
from PIL import ImageGrab
from numpy import ones, vstack
from numpy.linalg import lstsq
from driveKeys import PressKey, ReleaseKey, W, A, S, D
from statistics import mean

import warnings
# Ignore all warnings and never print.
warnings.filterwarnings('ignore')

In [2]:
def region_of_interest(frame, polygon):
    
    mask = np.zeros_like(frame)   
    cv2.fillPoly(mask, polygon, 255)
    
    # Returning the frame where mask pixels are non-zero
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

In [3]:
def draw_lanes(frame, lines):

    try:

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1], ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx, i in enumerate(lines):
            for xyxy in i:
                x_coords = (xyxy[0], xyxy[2])
                y_coords = (xyxy[1], xyxy[3])
                A = vstack([x_coords, ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                x1 = (min_y - b) / m
                x2 = (max_y - b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m, b, line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m, b, line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key = lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
                
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    
    except Exception as e:
        pass

In [4]:
def bgr_to_hls(frame):
    
    hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
      
    # defining range for Yellow and White Color
    lower = np.array([0, 153, 153])
    upper = np.array([255, 255, 255])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hls_frame, lower, upper)
    
    # Bitwise-AND mask with original frame
    hls_frame_white_yellow = cv2.bitwise_and(frame, frame, mask = mask)
    
    return hls_frame_white_yellow

In [5]:
def contrast_brightness(frame):
    
    contrast_brightness_frame = np.zeros(frame.shape, frame.dtype)
    alpha = 0   # Simple contrast control
    beta = 1    # Simple brightness control
       
    contrast_brightness_frame = cv2.convertScaleAbs(frame, alpha, beta)
    
    return contrast_brightness_frame

In [6]:
def image_processing(frame):
    
    original_frame = frame
    contrast_brightness_frame = contrast_brightness(frame)
    
    hls_frame = bgr_to_hls(contrast_brightness_frame)
    hls_to_bgr_frame = cv2.cvtColor(hls_frame, cv2.COLOR_HLS2BGR)
    
    gray_frame = cv2.cvtColor(hls_to_bgr_frame, cv2.COLOR_BGR2GRAY)
    canny_frame = cv2.Canny(gray_frame, 200, 300)    
    processed_frame = cv2.GaussianBlur(canny_frame, (5, 5), 0)
    
    polygon = np.array([[100, 450], [300, 200], [500, 200], [700, 450], [500, 450], [400, 300], [300, 450]], np.int32)
    processed_frame = region_of_interest(processed_frame, [polygon])
          
    lines = cv2.HoughLinesP(processed_frame, 1, np.pi/180, 180, np.array([]), 50, 10)
    
    m1, m2 = 0, 0
    try:        
        l1, l2, m1, m2 = draw_lanes(original_frame, lines)
        cv2.line(original_frame, (l1[0], l1[1]), (l1[2], l1[3]), [0, 255, 0], 20)
        cv2.line(original_frame, (l2[0], l2[1]), (l2[2], l2[3]), [0, 255, 0], 20)
    
    except Exception as e:
        pass
    
    try:        
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_frame, (coords[0], coords[1]), (coords[2], coords[3]), [255, 0, 0], 3)
              
            except Exception as e:
                pass
                
    except Exception as e:
        pass

    return hls_to_bgr_frame, processed_frame, original_frame, m1, m2

In [7]:
# Movement
def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    time.sleep(0.06)
    ReleaseKey(W)

def left():
    PressKey(W)
    PressKey(A)
    time.sleep(0.01)
    ReleaseKey(W)
    #ReleaseKey(A)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(D)
    

def right():
    PressKey(W)
    PressKey(D)
    time.sleep(0.01)
    ReleaseKey(W)
    #ReleaseKey(D)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(A)    
    
def slow():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    time.sleep(0.03)
    ReleaseKey(W)

In [8]:
def main():
    
    for i in list(range(5))[::-1]:
        print(i+1)
        time.sleep(1)
    
    last_time = time.time()
    while True:
        frame =  np.array(ImageGrab.grab(bbox = (0,40,800,630)))
        #print('Frame took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
        hls_to_bgr_frame, processed_frame, original_frame, m1, m2 = image_processing(frame)
        
        hls_to_bgr_frame = cv2.resize(hls_to_bgr_frame, (550, 400))
        processed_frame = cv2.resize(processed_frame, (550, 400))
        original_frame = cv2.resize(frame, (550, 400))
                
        cv2.imshow('HLS Frame', hls_to_bgr_frame)
        cv2.imshow('Lane Lines', processed_frame)
        cv2.imshow('Weighted Frame', cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB))
              
        if m1 < 0 and m2 < 0:
            right()
            print('Right    : [ {:.4f}, {:.4f} ]'.format(m1, m2))            
        elif m1 > 0 and m2 > 0:
            left()
            print('Left     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        elif m1 < 0 and m2 > 0:
            straight()
            print('Straight : [  {:.4f}, {:.4f} ]'.format(m1, m2)) 
        else:
            slow()
            print('Slow     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [9]:
main()

5
4
3
2
1
Left     : [  0.5544,  1.0000 ]
Left     : [  1.0000,  0.5759 ]
Left     : [  0.5765,  1.0000 ]
Left     : [  0.5548,  1.0000 ]
Left     : [  1.0000,  0.0534 ]
Left     : [  0.5326,  1.0000 ]
Left     : [  0.5543,  1.0000 ]
Left     : [  1.0000,  0.5776 ]
Left     : [  1.0000,  0.5109 ]
Left     : [  1.0000,  0.5539 ]
Left     : [  1.0000,  0.5551 ]
Left     : [  1.0000,  0.5560 ]
Left     : [  1.0000,  0.0952 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  1.0000,  1.1522 ]
Left     : [  1.0000,  0.5326 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  0.6746,  -1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Slow     : [  0.6987,  -1.0000 ]
Straight : [  -1.0000, 0.5538 ]
Straight : [  -1.0000, 0.6211 ]
Straight : [  -1.00

Straight : [  -1.0000, 0.2481 ]
Straight : [  -1.0000, 0.2310 ]
Straight : [  -1.0000, 0.2308 ]
Straight : [  -1.0000, 0.2299 ]
Straight : [  -1.0000, 0.2652 ]
Slow     : [  0.2305,  -1.0000 ]
Slow     : [  0.2492,  -1.0000 ]
Slow     : [  0.2655,  -1.0000 ]
Slow     : [  0.2500,  -1.2734 ]
Slow     : [  0.2492,  -1.0000 ]
Slow     : [  0.2708,  -1.2273 ]
Left     : [  0.2671,  0.6229 ]
Slow     : [  0.2703,  -1.0000 ]
Left     : [  0.2911,  0.7243 ]
Left     : [  0.2741,  0.8395 ]
Slow     : [  0.8381,  -1.0000 ]
Slow     : [  0.3059,  -0.8105 ]
Straight : [  -0.7582, 1.0000 ]
Slow     : [  1.0000,  -0.8125 ]
Left     : [  1.0000,  0.3271 ]
Straight : [  -1.0000, 1.0000 ]
Left     : [  0.3247,  1.0000 ]
Straight : [  -1.0000, 0.3101 ]
Slow     : [  1.0000,  -1.0000 ]
Straight : [  -1.0000, 0.7069 ]
Straight : [  -1.0000, 0.7534 ]
Straight : [  -1.0000, 0.7808 ]
Straight : [  -1.0000, 0.7519 ]
Straight : [  -1.0000, 0.7557 ]
Straight : [  -1.0000, 0.2692 ]
Straight : [  -1.0000, 0.2846

Straight : [  -1.0000, 0.7552 ]
Straight : [  -1.0000, 0.7509 ]
Straight : [  -1.0000, 0.7500 ]
Straight : [  -1.0000, 0.7517 ]
Straight : [  -0.8964, 0.7273 ]
Straight : [  -1.0000, 0.7826 ]
Slow     : [  0.7525,  -0.9016 ]
Slow     : [  0.7551,  -0.9055 ]
Straight : [  -0.8726, 0.6508 ]
Slow     : [  0.7809,  -0.7826 ]
Straight : [  -0.8717, 0.7809 ]
Straight : [  -0.8673, 1.0000 ]
Straight : [  -0.9024, 1.0000 ]
Straight : [  -0.8743, 0.4689 ]
Left     : [  0.4032,  1.0000 ]
Left     : [  0.4014,  1.0000 ]
Left     : [  0.4032,  1.0000 ]
Slow     : [  1.1510,  -0.7570 ]
Straight : [  -0.5744, 1.4797 ]
Straight : [  -0.5542, 1.4797 ]
Straight : [  -0.5770, 1.3250 ]
Straight : [  -0.6227, 1.4286 ]
Straight : [  -0.6258, 1.3766 ]
Straight : [  -0.6755, 1.1899 ]
Right    : [ -0.6735, -0.0692 ]
Right    : [ -0.6026, -0.0723 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.7823, 1.0000 ]
Slow     : [  0.0000,  0.0000 ]
Right    : [ -1.0000, -0.4853 ]
Slow

Slow     : [  0.4649,  -1.2362 ]
Slow     : [  0.5314,  -1.0000 ]
Straight : [  -1.0000, 0.6506 ]
Straight : [  -1.0000, 0.5327 ]
Straight : [  -1.0000, 0.6021 ]
Straight : [  -1.0000, 0.6244 ]
Left     : [  0.6005,  0.5118 ]
Straight : [  -1.0000, 0.2315 ]
Left     : [  0.6738,  0.6996 ]
Straight : [  -1.0000, 0.6490 ]
Straight : [  -1.0000, 0.6982 ]
Straight : [  -1.0000, 0.8036 ]
Slow     : [  1.0000,  -1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  0.4669,  -1.0000 ]
Left     : [  0.5089,  1.0000 ]
Slow     : [  0.5556,  -1.0000 ]
Straight : [  -1.0000, 0.5766 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  0.6236,  0.5345 ]
Left     : [  1.0000,  1.0000 ]
Left     : [  1.0000,  1.0000 ]
Slow     : [  1.0000,  -1.0000 ]
Slow     : [  1.0000,  -0.4909 ]
Slow     : [  1.0000,  -0.7273 ]
Straight : [  -0.6756, 1.0000 ]
Slow     : [  1.0000,  -0.7269 ]
Straight : [  -0.7529, 1.0000

Slow     : [  0.8373,  -0.0349 ]
Slow     : [  0.7786,  -1.0000 ]
Slow     : [  0.8410,  -1.0000 ]
Right    : [ -1.0000, -0.0541 ]
Straight : [  -1.0000, 0.4670 ]
Left     : [  0.4882,  0.8719 ]
Left     : [  0.4892,  0.7828 ]
Left     : [  0.4795,  0.7807 ]
Left     : [  0.6506,  0.8387 ]
Straight : [  -1.0000, 1.0000 ]
Left     : [  1.1943,  1.0000 ]
Slow     : [  1.0000,  -0.7000 ]
Slow     : [  1.0000,  -0.7826 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  1.0000,  -0.7548 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  1.0000,  -0.7535 ]
Straight : [  -0.7808, 1.0000 ]
Straight : [  -0.7818, 1.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.7803, 1.0000 ]
Slow     : [  1.0000,  -0.8378 ]
Slow     : [  1.0000,  -0.7831 ]
Slow     : [  1.0000,  -0.7222 ]
Slow     : [  1.0000,  -0.5231 ]
Slow     : [  1.0000,  -0.808